In [0]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
pip uninstall tensorflow

Uninstalling tensorflow-1.15.0rc3:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0rc3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0rc3


In [0]:
pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 1.2MB/s 
     |████████████████████████████████| 3.8MB 33.3MB/s 
     |████████████████████████████████| 450kB 40.0MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
class CNN(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = tf.keras.layers.Conv2D(
      filters=32,
      kernel_size=[5, 5],
      padding='same',
      activation=tf.nn.relu
    )
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.conv2 = tf.keras.layers.Conv2D(
      filters = 64,
      kernel_size=[5, 5],
      padding='same',
      activation=tf.nn.relu
    )
    self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
    self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units=10)
    
    
  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    output = tf.nn.softmax(x)
    return output

In [0]:
class MNISTLoader():
  def __init__(self):
    mnist = tf.keras.datasets.mnist
    (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
    self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)
    self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)
    self.train_label = self.train_label.astype(np.int32)    # [60000]
    self.test_label = self.test_label.astype(np.int32)      # [10000]
    self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
  
  def get_batch(self, batch_size):
    index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
    return self.train_data[index, :], self.train_label[index]
    

In [0]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [0]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
  X, y = data_loader.get_batch(batch_size)
  with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
    loss = tf.reduce_mean(loss)
    print("batch %d: loss %f" % (batch_index, loss.numpy()))
  grads = tape.gradient(loss, model.variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.299668
batch 1: loss 2.264277
batch 2: loss 2.118947
batch 3: loss 1.939157
batch 4: loss 1.815939
batch 5: loss 1.724002
batch 6: loss 1.507288
batch 7: loss 1.185542
batch 8: loss 1.034210
batch 9: loss 1.050065
batch 10: loss 0.880952
batch 11: loss 0.788368
batch 12: loss 0.751042
batch 13: loss 0.635837
batch 14: loss 0.634929
batch 15: loss 0.466768
batch 16: loss 0.730706
batch 17: loss 0.517589
batch 18: loss 0.443110
batch 19: loss 0.663282
batch 20: loss 0.724766
batch 21: loss 0.220491
batch 22: loss 0.379553
batch 23: loss 0.619748
batch 24: loss 0.303945
batch 25: loss 0.227237
batch 26: loss 0.389726
batch 27: loss 0.469454
batch 28: loss 0.474745
batch 29: loss 0.164930
batch 30: loss 0.410860
batch 31: loss 0.365599
batch 32: loss 0.257282
batch 33: loss 0.253841
batch 34: loss 0.493567
batch 35: loss 0.117126
batch 36: loss 0.182203
batch 37: loss 0.369401
batch 38: loss 0.396155
batch 39: loss 0.422206
batch 40: loss 0.322732
batch 41: loss 0.376624
ba

In [0]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
  start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
  y_pred = model.predict(data_loader.test_data[start_index: end_index])
  sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index:end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.989800
